In [1]:
import snowflake.connector
import sys
import pandas as pd
import numpy as np
import re
import json
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import time
import ast

In [2]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 200
pd.set_option('display.width', 2000)

In [3]:
con = snowflake.connector.connect(
    user="tliang@endpointclosing.com",
    account="endpoint",
    authenticator="externalbrowser",
    role="SNOWFLAKE_DATA_ENGINEERING",
    warehouse="DATAENGINEERING_WH"
)
cur = con.cursor()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [4]:
# random sample big 4 state, 125 for them, make sure all of them have email and mobile phone
# imput the sample contact
# sample 500 of them to retrieve the data, using as many params as possible, evaluate: hit rate, missing values, data quality compare with the salesforce data
# using same 500 people, remove mobile phone and retrieve again to see the difference of hit rate
# one without email
# one without email or phone 

In [5]:
# make sure the params"include_if_matched" == True

In [6]:
# API KEY for testing: de79d656a450ad880f71b316ba6334e01c411be670cf832aac756293c2763008
# API KEY For Endpoint: ce8e4e0d9b819994f79cf0603d17b4997c46114d328d747f295b6f21f89deacd

# Prepare the Functions

In [53]:
def lead_samplier(input_cols:list, df:pd.DataFrame, n_sample:int, random_state:int) -> pd.DataFrame:

    #specify the name of columns we are interested
    search_colname = ["NAME","EMAIL","PHONE","MOBILE_PHONE","COMPANY","CITY","STATE"]

    # based on the input columns, create the complement list
    exclude_list = [e for e in search_colname if e not in input_cols]

    # if exist, filter out records and sample n number of them that are all null in complement, but all not null in input list, else it returns empty list
    record_df = df[(df[input_cols].notna().all(1))&(df[exclude_list].isna().all(1))]
    if len(record_df) != 0:
        sample_df = df[(df[input_cols].notna().all(1))&(df[exclude_list].isna().all(1))][["ID"] + search_colname].fillna("").sample(n= min(n_sample, len(record_df)), random_state= random_state).reset_index(drop = True)
    else:
        sample_df = pd.DataFrame()
    return sample_df

In [54]:
def pdl_matcher(spec_col_list:list, input_df:pd.DataFrame, api_key:str) -> pd.DataFrame:
# a function to parse the data using People Data Lab API
    if len(input_df) == 0:
        return input_df

    else:
        for col in input_df.columns:
            if col not in spec_col_list:
                input_df[col] = ""
                
        input_df['NAME'] = [i.lower() for i in input_df["NAME"]]

        API_KEY = api_key
        pdl_url= "https://api.peopledatalabs.com/v5/person/enrich"

        pdl_response = []
        for i in range(len(input_df)):
            params = {
                "api_key": API_KEY,
                "name": [input_df.loc[i,"NAME"]], 
                "email":[input_df.loc[i,"EMAIL"]],
                "phone":[input_df.loc[i,"PHONE"]+ input_df.loc[i,"MOBILE_PHONE"]],
                "company":[input_df.loc[i,"COMPANY"]],  
                "location": [input_df.loc[i,"CITY"]+ input_df.loc[i,"STATE"]],
                "min_likelihood": 1,
                "include_if_matched": True
                }
        
            json_response = requests.get(pdl_url,  params=params).json()
            pdl_response.append(json_response)
        
        input_df['PDL_RESPONSE'] = pdl_response

        output_df = pd.concat([input_df.iloc[:,:-1], pd.json_normalize(input_df["PDL_RESPONSE"])],axis= 1)
    
        return output_df

In [58]:
# for subset in lists
# use subset call lead_sampler function, output sample_df
# use sample_df to call pdl_matcher, output pdl_output
# add subset information to pdl_output, concat into overall_df
# output the overall_df

def pdl_parser(subsets_list:list, lead_df:pd.DataFrame, need_sample:bool, n_sample:int, api_key:str)-> pd.DataFrame:
# a wrap up function for parsing and concat the parsed data to lead/contact

    

    output_df = pd.DataFrame()

    for subset in subsets_list:

        if need_sample == True:
            sample_df = lead_samplier(subset,lead_df, n_sample=n_sample, random_state=1)
        else:
            sample_df = lead_df[["NAME","EMAIL","PHONE","MOBILE_PHONE","COMPANY","CITY","STATE"]].reset_index(drop = True).fillna("")
        
        for i in range(len(sample_df)):
            
            i_df = sample_df.iloc[i,:].to_frame().transpose().reset_index(drop = True)
            sample_df_pdl = pdl_matcher(subset, i_df, api_key = api_key)

            if len(sample_df_pdl) != 0:
                sample_df_pdl["SEARCH_SUBSET"] = str(subset)
        
            output_df = pd.concat([output_df,sample_df_pdl], ignore_index= True)
            
            # API limit is 100 record/ min
            time.sleep(1)
    
    output_df = output_df.reset_index(drop=True)

    return output_df

# Prepare the Data

In [20]:
sample_contact = pd.read_csv("../data/four_states_sample_contact.csv",index_col = 0)
sample_contact["STATE"] = ["WA" if i == "Washington" else i for i in sample_contact["MAILING_STATE"]]
sample_contact["CITY"] = sample_contact["MAILING_CITY"]

In [21]:
query = "SELECT ID, NAME FROM LAKE.SALESFORCE.ACCOUNT WHERE IS_DELETED = FALSE"
account_info = cur.execute(query).fetch_pandas_all().reset_index(drop = True).rename(columns = {"ID":"ACCOUNT_ID","NAME":"COMPANY"})

In [22]:
sample_contact = sample_contact.merge(account_info, on ="ACCOUNT_ID", how = "inner").reset_index(drop=True)

In [23]:
test_sample_contact = sample_contact.sample(n = 10, random_state= 1).reset_index(drop=True)

In [24]:
# check if all params are included
sample_contact[["NAME","EMAIL","PHONE","MOBILE_PHONE","COMPANY","CITY","STATE"]].head()

,NAME,EMAIL,PHONE,MOBILE_PHONE,COMPANY,CITY,STATE
0,Paige Gordon,paige.gordon@corcorangroup.com,(818) 661-4661,(805) 750-2105,"The Address, Inc",Agoura Hills,CA
1,Tami Jo Simon,tamijo@addressrealestate.com,(818) 661-4661,(818) 687-5697,"The Address, Inc",Agoura Hills,CA
2,Clark Harding,clark@clarkharding.com,707-541-6120,(707) 227-2122,HomeSmart Advantage Realty,Santa Rosa,CA
3,Nick Schneider,nick.schneider@compass.com,(310)241-3038,(310) 809-4875,Compass El Segundo,Torrance,CA
4,Luis Frias,desertchoice@live.com,760-863-5021,7602006360,Desert Choice Realty,Indio,CA


In [25]:
test_sample_contact.head()

,index,ID,IS_DELETED,MASTER_RECORD_ID,ACCOUNT_ID,IS_PERSON_ACCOUNT,LAST_NAME,FIRST_NAME,SALUTATION,MIDDLE_NAME,SUFFIX,NAME,RECORD_TYPE_ID,OTHER_STREET,OTHER_CITY,OTHER_STATE,OTHER_POSTAL_CODE,OTHER_COUNTRY,OTHER_LATITUDE,OTHER_LONGITUDE,OTHER_GEOCODE_ACCURACY,MAILING_STREET,MAILING_CITY,MAILING_STATE,MAILING_POSTAL_CODE,MAILING_COUNTRY,MAILING_LATITUDE,MAILING_LONGITUDE,MAILING_GEOCODE_ACCURACY,PHONE,FAX,MOBILE_PHONE,HOME_PHONE,OTHER_PHONE,ASSISTANT_PHONE,REPORTS_TO_ID,EMAIL,TITLE,DEPARTMENT,ASSISTANT_NAME,LEAD_SOURCE,BIRTHDATE,DESCRIPTION,OWNER_ID,HAS_OPTED_OUT_OF_EMAIL,HAS_OPTED_OUT_OF_FAX,DO_NOT_CALL,CREATED_DATE,CREATED_BY_ID,LAST_MODIFIED_DATE,LAST_MODIFIED_BY_ID,SYSTEM_MODSTAMP,LAST_ACTIVITY_DATE,LAST_CUREQUEST_DATE,LAST_CUUPDATE_DATE,LAST_VIEWED_DATE,LAST_REFERENCED_DATE,EMAIL_BOUNCED_REASON,EMAIL_BOUNCED_DATE,IS_EMAIL_BOUNCED,PHOTO_URL,JIGSAW,JIGSAW_CONTACT_ID,INDIVIDUAL_ID,TOTAL_MODUS_TRANSACTION_COUNT_C,CUSTOMER_STATUS_C,MOST_RECENT_CAMPAIGN_C,PRODUCT_DEMO_DATE_C,ROLE_C,KCA_C,LISTING_PRICE_C,KNOCK_WEBHOOK_C,CONVERTED_FROM_LEAD_C,ALTERNATE_EMAIL_C,FA_LOYAL_LEAD_C,PREFERRED_ESCROW_COMPANY_C,PREFERRED_TITLE_COMPANY_C,TOTAL_SALES_C,TOTAL_LISTING_BOOK_SIZE_C,TOTAL_LISTING_COUNT_C,TOTAL_TRANSACTION_COUNT_C,STATUS_C,APP_DOWNLOAD_DATE_C,FSBO_ADDRESS_C,ORDER_ENTRY_NOTES_C,FSBO_LISTING_PRICE_C,EMAIL_BOUNCED_C,FIRST_TASK_DATE_C,FIRST_EVENT_DATE_C,SALES_LOFT_1_MOST_RECENT_CADENCE_NAME_C,DISQUALIFIED_REASONS_C,SALES_LOFT_1_MOST_RECENT_CADENCE_NEXT_STEP_DUE_DATE_C,SALES_LOFT_1_MOST_RECENT_LAST_COMPLETED_STEP_C,EVENT_TYPE_C,PREFERRED_PLATFORM_C,CHECKBOX_C,NMLS_ID_C,REFERRER_C,COMING_SOON_ADDRESS_C,TOTAL_DIRECTED_ENDPOINT_ORDERS_C,PI_NEEDS_SCORE_SYNCED_C,PI_PARDOT_LAST_SCORED_AT_C,PI_CAMPAIGN_C,PI_COMMENTS_C,PI_CONVERSION_DATE_C,PI_CONVERSION_OBJECT_NAME_C,PI_CONVERSION_OBJECT_TYPE_C,PI_CREATED_DATE_C,PI_FIRST_ACTIVITY_C,PI_FIRST_SEARCH_TERM_C,PI_FIRST_SEARCH_TYPE_C,PI_FIRST_TOUCH_URL_C,PI_GRADE_C,PI_LAST_ACTIVITY_C,PI_NOTES_C,PI_PARDOT_HARD_BOUNCED_C,PI_SCORE_C,PI_URL_C,PI_UTM_CAMPAIGN_C,PI_UTM_CONTENT_C,PI_UTM_MEDIUM_C,PI_UTM_SOURCE_C,PI_UTM_TERM_C,AGENT_LICENSE_C,AGENT_MLS_ID_C,AVERAGE_CLOSE_PRICE_C,BS_COUNT_C,BUYER_SELLER_LINK_C,COUNTY_C,TOTAL_NON_DIRECTED_ENDPOINT_ORDERS_C,FA_DIRECTED_ESCROW_TRANSACTIONS_C,FA_DIRECTED_TRANSACTIONS_C,LEAD_RANKING_C,LEVEL_C,LS_SALES_C,LISTING_COUNT_C,MATRIX_UNIQUE_ID_C,PMX_AGENT_ID_C,PREFERRED_ESCROW_CO_C,PREFERRED_ESCROW_SOW_C,PREFERRED_TITLE_SOW_C,SECONDARY_ESCROW_CO_C,SECONDARY_TITLE_CO_C,SIGN_UP_DATE_C,SUPPORT_TEAM_C,BS_SALES_C,TOTAL_DIRECTED_ESCROW_TRANSACTIONS_C,TOTAL_DIRECTED_TITLE_TRANSACTIONS_C,PREFERRED_TITLE_CO_C,ALTERNATE_PHONE_C,_FIVETRAN_SYNCED,DISQUALIFIED_NOTES_C,RETIRING_C,PREFERS_TRADITIONAL_C,NO_CONTACT_C,IN_HOUSE_ESCROW_JV_C,BAD_EXPERIENCE_C,NO_DIRECTING_ORDERS_C,UP_TO_BROKER_C,EMAIL_UNKNOWN_DELIVERABILITY_C,ACCOUNT_MANAGER_C,SALES_DEVELOPMENT_REP_C,RELATIONSHIP_MANAGER_C,WEB_LEAD_C,DO_NOT_CONTACT_C,DEMO_SET_ON_C,INACTIVE_C,CONVERTED_LEAD_ID_C,DEMO_SCHEDULED_FOR_C,FIRST_OPEN_ORDER_DATE_C,LAST_OPEN_ORDER_DATE_C,DEMO_NOT_HELD_C,ONBOARDED_DATE_C,TERRITORY_C,PRODUCT_RE_DEMO_DATE_C,WIN_BACK_CLIENT_C,CANCELLED_FILE_C,INACTIVITY_C,TRANSFERRED_BDM_C,JET_CLOSING_C,NO_SHOW_CP_C,MEETING_TYPE_CP_C,BOOKING_STATUS_CP_C,RECENT_CONTACT,RECENT_CONTACT_QUARTER,MAILING_STATE_STD,STATE,CITY,COMPANY
0,19,0031H00002SqoKTQAZ,False,NaN,0011H00002AJjd5QAD,False,Urias dupont,Ana,NaN,Claudia,NaN,Ana Claudia Urias dupont,0121H000000zyGqQAI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7333 E DOUBLETREE RANCH RD STE 100,Scottsdale,AZ,85258,NaN,NaN,NaN,NaN,(480) 998-0110,NaN,(602) 643-8831,NaN,NaN,NaN,NaN,anaurias0514@gmail.com,NaN,NaN,NaN,Growth Campaign,NaN,NaN,0051H00000BeZYAQA3,False,False,False,2021-07-06 14:20:44-07:00,0051H00000BemggQAB,2022-05-26 11:26:07-07:00,0051H00000BeZYAQA3,2022-05-26 11:26:07-07:00,2022-05-26,NaN,NaN,NaN,NaN,NaN,NaN,False,/services/images/photo/0031H00002SqoKTQAZ,NaN,NaN,NaN,NaN,Committed,NaN,NaN,Agent,False,NaN,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,Qualified,NaN,NaN,NaN,NaN,False,20

# Test the query

In [26]:
# test_sample_subset = [["NAME","EMAIL","PHONE","MOBILE_PHONE","COMPANY","CITY","STATE"]]
# test_api_key = "de79d656a450ad880f71b316ba6334e01c411be670cf832aac756293c2763008"
# retrieve_test_sample_contact = pdl_parser(subsets_list = test_sample_subset,
#                                         lead_df = test_sample_contact,
#                                         need_sample = False,
#                                         n_sample= 10,
#                                         api_key= test_api_key)

In [27]:
# test_sample_subset2 = [["NAME","PHONE","COMPANY","CITY","STATE"]]
# test_api_key = "de79d656a450ad880f71b316ba6334e01c411be670cf832aac756293c2763008"
# retrieve_test_sample_contact2 = pdl_parser(subsets_list = test_sample_subset2,
#                                         lead_df = test_sample_contact,
#                                         need_sample = False,
#                                         n_sample= 10,
#                                         api_key= test_api_key)

In [59]:
# test_sample_subset3 = [["NAME","EMAIL","PHONE","MOBILE_PHONE","COMPANY","CITY","STATE"]]
# test_api_key = "ce8e4e0d9b819994f79cf0603d17b4997c46114d328d747f295b6f21f89deacd"
# retrieve_test_sample_contact3= pdl_parser(subsets_list = test_sample_subset3,
#                                         lead_df = test_sample_contact.sample(n = 5, random_state=1),
#                                         need_sample = False,
#                                         n_sample= 5,
#                                         api_key= test_api_key)

In [60]:
retrieve_test_sample_contact3

,NAME,EMAIL,PHONE,MOBILE_PHONE,COMPANY,CITY,STATE,status,likelihood,matched,dataset_version,data.id,data.full_name,data.first_name,data.middle_initial,data.middle_name,data.last_initial,data.last_name,data.gender,data.birth_year,data.birth_date,data.linkedin_url,data.linkedin_username,data.linkedin_id,data.facebook_url,data.facebook_username,data.facebook_id,data.twitter_url,data.twitter_username,data.github_url,data.github_username,data.work_email,data.personal_emails,data.mobile_phone,data.industry,data.job_title,data.job_title_role,data.job_title_sub_role,data.job_title_levels,data.job_company_id,data.job_company_name,data.job_company_website,data.job_company_size,data.job_company_founded,data.job_company_industry,data.job_company_linkedin_url,data.job_company_linkedin_id,data.job_company_facebook_url,data.job_company_twitter_url,data.job_company_location_name,data.job_company_location_locality,data.job_company_location_metro,data.job_company_location_region,data.job_company_location_geo,data.job_company_location_street_address,data.job_company_location_address_line_2,data.job_company_location_postal_code,data.job_company_location_country,data.job_company_location_continent,data.job_last_updated,data.job_start_date,data.location_name,data.location_locality,data.location_metro,data.location_region,data.location_country,data.location_continent,data.location_street_address,data.location_address_line_2,data.location_postal_code,data.location_geo,data.location_last_updated,data.phone_numbers,data.emails,data.interests,data.skills,data.location_names,data.regions,data.countries,data.street_addresses,data.experience,data.education,data.profiles,data.version_status.status,data.version_status.contains,data.version_status.previous_version,data.version_status.current_version,SEARCH_SUBSET,error.type,error.message
0,jim stafford,j.stafford@ggsir.com,415-381-7300,(415) 706-6025,Golden Gate Sotheby's Int'l,Mill Valley,CA,200,10.0,"[email, name]",18.1,pMUgrxrkM7swc-eMz0-88g_0000,jim stafford,jim,None,None,s,stafford,male,None,None,None,None,None,None,None,None,None,None,None,None,None,[],None,None,None,None,None,[],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,[],"[{'address': 'j.stafford@ggsir.com', 'type': N...",[],[],[],[],[],[],[{'company': {'name': 'decker bullock sotheby'...,[],[],,[],,,"['NAME', 'EMAIL', 'PHONE', 'MOBILE_PHONE', 'CO...",NaN,NaN
1,marcelo marcondes,marceloagent@gmail.com,(415) 897-3000,(415) 497-5227,Coldwell Banker Realty,Novato,CA,200,10.0,"[email, company, name]",18.1,REOfjB2tVzyeQsaoezt0Pg_0000,marcelo marcondes,marcelo,None,None,m,marcondes,male,None,None,linkedin.com/in/marcelomarcondes,marcelomarcondes,25331461,None,None,1215200362,twitter.com/marceloagent,marceloagent,None,None,None,"[marcondes.realestate@gmail.com, marcondesreal...",None,real estate,real estate broker,finance,None,[],coldwell-banker,coldwell banker,coldwellbanker.com,10001+,1906,real estate,linkedin.com/company/coldwell-banker,2037,None,None,"madison, new jersey, united states",madison,"new york, new york",new jersey,"40.75,-74.41",175 park avenue,None,07940,united states,north america,2022-02-08,None,"san rafael, california, united states",san rafael,"san francisco, california",california,united states,north america,114 surrey lane,None,94903,"37.97,-122.53",2022-04-07,[],"[{'address': 'marcondes.realestate@gmail.com',...",[politics],"[real estate, investors, residential homes, fo...","[san rafael, california, united states, elk ri...","[california, united states, sao paulo, brazil,...","[brazil, united states]","[{'name': 'elk river, minnesota, united states...","[{'company': {'name': 'coldwell banker', 'size...",[{'school': {'name': 'american bartending scho...,"[{'network': 'linkedin', 'id': '25331461', 'ur...",,[],,,"['NAME', 'EMAIL', 'PHONE', 'MOBILE_PHONE', 'CO...",NaN,NaN
2,brian edward eastwood,brian@brianeastwood.

In [28]:
retrieve_test_sample_contact

,NAME,EMAIL,PHONE,MOBILE_PHONE,COMPANY,CITY,STATE,status,likelihood,matched,dataset_version,data.id,data.full_name,data.first_name,data.middle_initial,data.middle_name,data.last_initial,data.last_name,data.gender,data.birth_year,data.birth_date,data.linkedin_url,data.linkedin_username,data.linkedin_id,data.facebook_url,data.facebook_username,data.facebook_id,data.twitter_url,data.twitter_username,data.github_url,data.github_username,data.work_email,data.personal_emails,data.mobile_phone,data.industry,data.job_title,data.job_title_role,data.job_title_sub_role,data.job_title_levels,data.job_company_id,data.job_company_name,data.job_company_website,data.job_company_size,data.job_company_founded,data.job_company_industry,data.job_company_linkedin_url,data.job_company_linkedin_id,data.job_company_facebook_url,data.job_company_twitter_url,data.job_company_location_name,data.job_company_location_locality,data.job_company_location_metro,data.job_company_location_region,data.job_company_location_geo,data.job_company_location_street_address,data.job_company_location_address_line_2,data.job_company_location_postal_code,data.job_company_location_country,data.job_company_location_continent,data.job_last_updated,data.job_start_date,data.location_name,data.location_locality,data.location_metro,data.location_region,data.location_country,data.location_continent,data.location_street_address,data.location_address_line_2,data.location_postal_code,data.location_geo,data.location_last_updated,data.phone_numbers,data.emails,data.interests,data.skills,data.location_names,data.regions,data.countries,data.street_addresses,data.experience,data.education,data.profiles,data.version_status.status,data.version_status.contains,data.version_status.previous_version,data.version_status.current_version,error.type,error.message,SEARCH_SUBSET
0,ana claudia urias dupont,anaurias0514@gmail.com,(480) 998-0110,(602) 643-8831,HAGUE PARTNERS,Scottsdale,AZ,200,7.0,"[email, first_name]",18.1,0ujlRs-i13oSTEGigegFnA_0000,ana urias,ana,None,NaN,u,urias,female,NaN,NaN,None,None,None,None,None,None,None,None,NaN,NaN,None,[anaurias0514@gmail.com],None,None,None,None,None,[],None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"phoenix, arizona, united states",phoenix,"phoenix, arizona",arizona,united states,north america,None,None,85029,"33.44,-112.07",None,[+16026438831],"[{'address': 'anaurias0514@gmail.com', 'type':...",[],[],"[phoenix, arizona, united states]","[arizona, united states]",[united states],[],[],[],[],,[],,,NaN,NaN,"['NAME', 'EMAIL', 'PHONE', 'MOBILE_PHONE', 'CO..."
1,basman zahrah,basman@westusa.com,(480) 893-0600,(602) 748-8642,West Usa Realty,Phoenix,AZ,200,9.0,"[company, name]",18.1,PRMwwFrPELwJg7IZKdG8JQ_0000,basman zahrah,basman,None,NaN,z,zahrah,None,NaN,NaN,linkedin.com/in/basman-zahrah-078275a5,basman-zahrah-078275a5,373899548,None,None,None,None,None,NaN,NaN,None,[],None,real estate,real estate agent,real_estate,realtor,[],west-usa-realty,west usa realty,joinwestusa.com,1001-5000,1986.0,real estate,linkedin.com/company/west-usa-realty,376195,None,None,"phoenix, arizona, united states",phoenix,"phoenix, arizona",arizona,"33.44,-112.07",2355 west utopia road,None,85027,united states,north america,2022-04-08,2017-07,"phoenix, arizona, united states",phoenix,"phoenix, arizona",arizona,united states,north america,None,None,None,"33.44,-112.07",2022-04-08,[],[],[science and technology],"[autocad, microsoft office, mechanics, enginee...","[phoenix, arizona, united states]","[arizona, united states]",[united states],[],[{'company': {'name': 'cab comm cox contractor...,"[{'school': {'name': 'jami'at al - mosul', 'ty...","[{'network': 'linkedin', 'id': '373899548', 'u...",,[],,,NaN,NaN,"['NAME', 'EMAIL', 'PHONE', 'MOBILE_PHONE', 'CO..."
2,jim stafford,j.stafford@ggsir.com,415-381-7300,(415) 706-6025,Golden Gate Sotheby's Int'l,Mill Valley,CA,200,10.0,"[email, name]",18.1,pMUgrxrkM7swc-eMz0-88g_0000,jim sta

In [29]:
retrieve_test_sample_contact2

,NAME,EMAIL,PHONE,MOBILE_PHONE,COMPANY,CITY,STATE,status,likelihood,matched,dataset_version,data.id,data.full_name,data.first_name,data.middle_initial,data.middle_name,data.last_initial,data.last_name,data.gender,data.birth_year,data.birth_date,data.linkedin_url,data.linkedin_username,data.linkedin_id,data.facebook_url,data.facebook_username,data.facebook_id,data.twitter_url,data.twitter_username,data.github_url,data.github_username,data.work_email,data.personal_emails,data.mobile_phone,data.industry,data.job_title,data.job_title_role,data.job_title_sub_role,data.job_title_levels,data.job_company_id,data.job_company_name,data.job_company_website,data.job_company_size,data.job_company_founded,data.job_company_industry,data.job_company_linkedin_url,data.job_company_linkedin_id,data.job_company_facebook_url,data.job_company_twitter_url,data.job_company_location_name,data.job_company_location_locality,data.job_company_location_metro,data.job_company_location_region,data.job_company_location_geo,data.job_company_location_street_address,data.job_company_location_address_line_2,data.job_company_location_postal_code,data.job_company_location_country,data.job_company_location_continent,data.job_last_updated,data.job_start_date,data.location_name,data.location_locality,data.location_metro,data.location_region,data.location_country,data.location_continent,data.location_street_address,data.location_address_line_2,data.location_postal_code,data.location_geo,data.location_last_updated,data.phone_numbers,data.emails,data.interests,data.skills,data.location_names,data.regions,data.countries,data.street_addresses,data.experience,data.education,data.profiles,data.version_status.status,data.version_status.contains,data.version_status.previous_version,data.version_status.current_version,error.type,error.message,SEARCH_SUBSET
0,ana claudia urias dupont,,(480) 998-0110,,HAGUE PARTNERS,Scottsdale,AZ,200,7.0,"[phone, company]",18.1,lpVHBenXnQd1C-9sF8uMnw_0000,michael brown,michael,w,NaN,b,brown,male,NaN,None,linkedin.com/in/michael-w-brown-1b7a773,michael-w-brown-1b7a773,11603803,facebook.com/michael.washingtonbrown,michael.washingtonbrown,100003886386571,None,None,NaN,NaN,NaN,[],None,real estate,real estate agent,real_estate,realtor,[],None,brown residential group,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,2018-12-01,2013-03,"phoenix, arizona, united states",phoenix,"phoenix, arizona",arizona,united states,north america,None,None,None,"33.44,-112.07",2018-12-01,[+14809980110],"[{'address': 'michael@paradisevalley.com', 'ty...","[working out, creative writing, racquetball, s...","[foreclosures, yoga, single family homes, real...","[phoenix, arizona, united states]","[arizona, united states]",[united states],[],[{'company': {'name': 'brown residential group...,[{'school': {'name': 'sir ernest bevin school'...,"[{'network': 'linkedin', 'id': '11603803', 'ur...",,[],,,NaN,NaN,"['NAME', 'PHONE', 'COMPANY', 'CITY', 'STATE']"
1,basman zahrah,,(480) 893-0600,,West Usa Realty,Phoenix,AZ,200,9.0,"[company, name]",18.1,PRMwwFrPELwJg7IZKdG8JQ_0000,basman zahrah,basman,None,NaN,z,zahrah,None,NaN,None,linkedin.com/in/basman-zahrah-078275a5,basman-zahrah-078275a5,373899548,None,None,None,None,None,NaN,NaN,NaN,[],None,real estate,real estate agent,real_estate,realtor,[],west-usa-realty,west usa realty,joinwestusa.com,1001-5000,1986.0,real estate,linkedin.com/company/west-usa-realty,376195,None,None,"phoenix, arizona, united states",phoenix,"phoenix, arizona",arizona,"33.44,-112.07",2355 west utopia road,NaN,85027,united states,north america,2022-04-08,2017-07,"phoenix, arizona, united states",phoenix,"phoenix, arizona",arizona,united states,north america,None,None,None,"33.44,-112.07",2022-04-08,[],[],[science and technology],"[autocad, microsoft office, mechanics, enginee...","[phoenix, arizona, united states]","[arizona, united states]",[united states],[],[{'company': {'name': 'cab comm cox contractor...,"[{'school': {

# Formal Retrieve

## Case 1: all params

In [67]:
# pdl_contact_all_params = pd.read_csv("../data/pdl_data/sample_contact_pdl_all_params.csv")
# pdl_contact_all_params_to_retrieve = pdl_contact_all_params[pdl_contact_all_params['status'] == 402][["NAME","EMAIL","PHONE","MOBILE_PHONE","COMPANY","CITY","STATE"]]

In [69]:
# # for retrieve the rest of the records
# # case 1: retrieve data using all params
# sample_subset = [["NAME","EMAIL","PHONE","MOBILE_PHONE","COMPANY","CITY","STATE"]]
# ep_api_key = "ce8e4e0d9b819994f79cf0603d17b4997c46114d328d747f295b6f21f89deacd" # use test api in this case
# to_retrieve_sample_contact_pdl_all_params = pdl_parser(subsets_list = sample_subset,
#                                         lead_df = pdl_contact_all_params_to_retrieve,
#                                         need_sample = False,
#                                         n_sample= 500,
#                                         api_key= ep_api_key)

In [71]:
# to_retrieve_sample_contact_pdl_all_params['status'].value_counts()

200    279
404    133
Name: status, dtype: int64

In [72]:
# new_sample_contact_pdl_all_params = pd.concat([pdl_contact_all_params[pdl_contact_all_params['status'] != 402],to_retrieve_sample_contact_pdl_all_params])

In [76]:
# new_sample_contact_pdl_all_params.to_csv("../data/pdl_data/new_sample_contact_pdl_all_params.csv",index= False)

In [33]:
# # case 1: retrieve data using all params
# sample_subset = [["NAME","EMAIL","PHONE","MOBILE_PHONE","COMPANY","CITY","STATE"]]
# ep_api_key = "de79d656a450ad880f71b316ba6334e01c411be670cf832aac756293c2763008" # use test api in this case
# sample_contact_pdl_all_params = pdl_parser(subsets_list = sample_subset,
#                                         lead_df = sample_contact,
#                                         need_sample = False,
#                                         n_sample= 500,
#                                         api_key= ep_api_key)

In [34]:
# sample_contact_pdl_all_params.to_csv("../data/pdl_data/sample_contact_pdl_all_params.csv",index= False)

## Case 2: without mobile phone

In [79]:
# pdl_contact_no_mobile_phone = pd.read_csv("../data/pdl_data/sample_contact_pdl_no_mobile_phone.csv")
# pdl_contact_no_mobile_phone_to_retrieve = pdl_contact_no_mobile_phone[pdl_contact_no_mobile_phone['status'] == 429][["NAME","EMAIL","PHONE","MOBILE_PHONE","COMPANY","CITY","STATE"]]

In [80]:
# # case 2: without mobile phone
# sample_subset = [["NAME","EMAIL","PHONE","COMPANY","CITY","STATE"]]
# ep_api_key = "ce8e4e0d9b819994f79cf0603d17b4997c46114d328d747f295b6f21f89deacd"
# to_retrieve_sample_contact_pdl_no_mobile_phone = pdl_parser(subsets_list = sample_subset,
#                                         lead_df = pdl_contact_no_mobile_phone_to_retrieve,
#                                         need_sample = False,
#                                         n_sample= 500,
#                                         api_key= ep_api_key)

In [81]:
# to_retrieve_sample_contact_pdl_no_mobile_phone['status'].value_counts()

200    169
404     36
Name: status, dtype: int64

In [82]:
# new_sample_contact_pdl_no_mobile_phone = pd.concat([pdl_contact_no_mobile_phone[pdl_contact_no_mobile_phone['status'] != 429],to_retrieve_sample_contact_pdl_no_mobile_phone])

In [85]:
# new_sample_contact_pdl_no_mobile_phone.to_csv("../data/pdl_data/new_sample_contact_pdl_no_mobile_phone.csv",index= False)

In [37]:
# # case 2: without mobile phone
# sample_subset = [["NAME","EMAIL","PHONE","COMPANY","CITY","STATE"]]
# ep_api_key = "ce8e4e0d9b819994f79cf0603d17b4997c46114d328d747f295b6f21f89deacd" 
# sample_contact_pdl_no_mobile_phone = pdl_parser(subsets_list = sample_subset,
#                                         lead_df = sample_contact,
#                                         need_sample = False,
#                                         n_sample= 500,
#                                         api_key= ep_api_key)

In [39]:
# sample_contact_pdl_no_mobile_phone.to_csv("../data/pdl_data/sample_contact_pdl_no_mobile_phone.csv",index= False)

## Case 3: without email

In [90]:
# # case 3: without email
# sample_subset = [["NAME","PHONE","MOBILE_PHONE","COMPANY","CITY","STATE"]]
# ep_api_key = "ce8e4e0d9b819994f79cf0603d17b4997c46114d328d747f295b6f21f89deacd"
# sample_contact_pdl_no_email = pdl_parser(subsets_list = sample_subset,
#                                         lead_df = sample_contact,
#                                         need_sample = False,
#                                         n_sample= 500,
#                                         api_key= ep_api_key)

In [91]:
# sample_contact_pdl_no_email['status'].value_counts()

404    275
200    225
Name: status, dtype: int64

In [93]:
# sample_contact_pdl_no_email.to_csv("../data/pdl_data/new_sample_contact_pdl_no_email.csv",index= False)

## Case 4: without email or phone

In [95]:
# # case 4: without email and mobile phone
# sample_subset = [["NAME","PHONE","COMPANY","CITY","STATE"]]
# ep_api_key = "ce8e4e0d9b819994f79cf0603d17b4997c46114d328d747f295b6f21f89deacd"
# sample_contact_pdl_no_email_and_mobile_phone = pdl_parser(subsets_list = sample_subset,
#                                         lead_df = sample_contact,
#                                         need_sample = False,
#                                         n_sample= 500,
#                                         api_key= ep_api_key)

In [97]:
# sample_contact_pdl_no_email_and_mobile_phone.to_csv("../data/pdl_data/new_sample_contact_pdl_no_email_and_mobile_phone.csv",index= False)